## Nocturne concepts

This page introduces the most basic elements of nocturne. You can find further information about these [in Section 3 of the paper](https://arxiv.org/abs/2206.09889).

_Last update: April 2023_

In [1]:
import numpy as np

data_path = 'data/example_scenario.json'

### Summary

- Nocturne simulations are discretized traffic scenarios. A scenario is a constructed snapshot of traffic situation at a particular timepoint.
- Vehicles observe the traffic scene from their own viewpoint and therefore the state of every vehicle is unique. We call the state of the vehicle we focus on the ego state.
- Nocturne incorporates physical principles in rendering objects in the traffic scene. `#TODO`

### Simulation

In Nocturne, a simulation discretizes an existing traffic scenario. At the moment, Nocturne supports traffic scenarios from the Waymo Open Dataset, but can be further extended to work with other driving datasets. 

<figure>
<center>
<img src='https://drive.google.com/uc?id=1nv5Rbyf7ZfdqTdaUduXvEI7ncdkLpDjc' width='600'/>
<figcaption></figcaption>An example of a set of traffic scenario's in Nocturne. Upon initialization, a start time is chosen. After each iteration we take a step in the simulation, which gets us to the next scenario. This is done until we reach the end of the simulation (last image). </center>
</figure>

We show an example of this using `example_scenario.json`, where our traffic data is extracted from the Waymo open motion dataset:

In [4]:
from nocturne import Simulation

scenario_config = {
    'start_time': 0, # When to start the simulation
    'allow_non_vehicles': True, # Whether to include cyclists and pedestrians 
    'max_visible_road_points': 1, # Maximum number of road points for a vehicle
    'max_visible_objects': 1, # Maximum number of road objects for a vehicle
    'max_visible_traffic_lights': 1,
    'max_visible_stop_signs': 1,
}

# Create simulation
sim = Simulation(data_path, scenario_config)

### Scenario

A simulation consists of a set of scenarios. A scenario is a snapshot of the traffic scene at a particular timepoint. 

Here is how to create a scenario object:

In [5]:
# Get traffic scenario at timepoint
scenario = sim.getScenario()

The `scenario` objects holds information we are interested in. Here are a couple of examples:

In [6]:
# The number of road objects in the scene
len(scenario.getObjects())

33

In [7]:
# The road objects that moved at a particular timepoint
objects_that_moved = scenario.getObjectsThatMoved()

print(f'Total # moving objects: {len(objects_that_moved)}\n')
print(f'Object IDs of moving vehicles: \n {[obj.getID() for obj in objects_that_moved]} ')

Total # moving objects: 15

Object IDs of moving vehicles: 
 [0, 1, 2, 3, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32] 


In [8]:
# Number of road lines
len(scenario.road_lines())

128

In [9]:
scenario.getVehicles()[:5]

In [10]:
# Select all moving vehicles that move 
moving_vehicles = [obj for obj in scenario.getVehicles() if obj in objects_that_moved]

print(f'Found {len(moving_vehicles)} moving vehicles in scene: {[vehicle.getID() for vehicle in moving_vehicles]}')

Found 2 moving vehicles in scene: [3, 32]


#### Ego state

The **ego state** is an array with features that describe the current vehicle. This array holds the following information: 
- 0: length of ego vehicle
- 1: width of ego vehicle
- 2: speed of ego vehicle
- 3: distance to the goal position of ego vehicle
- 4: angle to the goal (target azimuth) 
- 5: desired heading at goal position
- 6: desired speed at goal position
- 7: current acceleration
- 8: current steering position
- 9: current head angle

In [11]:
# Select an arbitrary vehicle
ego_vehicle = moving_vehicles[0]

print(f'Selected vehicle # {ego_vehicle.getID()}')

# Get the state for ego vehicle
scenario.ego_state(ego_vehicle)

Selected vehicle # 3


array([ 4.4936213 ,  1.9770377 ,  0.07662283,  4.24219   , -0.05617166,
       -0.05909407,  1.6792779 ,  0.        ,  0.        ,  0.        ],
      dtype=float32)

#### Visible state

We use the ego vehicle state, together with a view distance (how far the vehicle can see) and a view angle to construct the **visible state**. The figure below shows this procedure for a simplified traffic scene. 

The visible state comprises four matrices:
- `stop_signs`: the visible stop signs of `(1, 13)`
- `traffic_lights`: the states for the traffic lights from the perspective of the ego driver.
- `road_points`: the observable road points (static elements in the scene)
- `objects`: the observable road objects (vehicles, pedestrians and cyclists)

<figure>
<center>
<img src='https://drive.google.com/uc?id=1fG43NvPCzaimmW99asRdB73qY-F4u-q0' width='700'/>
<figcaption>To investigate coordination under partial observability, agents in Nocturne can only see an obstructed view of their environment. In this simplified traffic scene, we construct the state for the red ego driver. Note that Nocturne assumes that stop signs can be viewed, even if they are behind another driver. </figcaption></center>
</figure>

In [12]:
# Define viewing distance, radius and head angle
view_distance = 80 
view_angle = np.radians(120) 
head_angle = 0

# Construct the visible state for ego vehicle
visible_state = scenario.visible_state(
    ego_vehicle, 
    view_dist=view_distance, 
    view_angle=view_angle,
    head_angle=head_angle,
)

visible_state.keys()

dict_keys(['stop_signs', 'traffic_lights', 'road_points', 'objects'])

In [13]:
# No visible stop signs
visible_state['stop_signs']

array([], shape=(0, 3), dtype=float32)

In [14]:
# Empty; are filtered out in this version of Nocturne
visible_state['traffic_lights']

array([], shape=(0, 12), dtype=float32)

In [15]:
# Max visible road points x 13 features
visible_state['road_points'].shape

(1, 13)

In [16]:
# Number of visible road objects x features 
visible_state['objects'].shape

(1, 13)

In [17]:
dimension = 0

for key, val in visible_state.items():
    print(f"Number of features for '{key}' = {val.shape[1]}")

    dimension += val.shape[0] * val.shape[1]

dimension

Number of features for 'stop_signs' = 3
Number of features for 'traffic_lights' = 12
Number of features for 'road_points' = 13
Number of features for 'objects' = 13


26

In [18]:
# We can also flatten the visible state
visible_state_flat = scenario.flattened_visible_state(
        ego_vehicle, view_dist=view_distance, view_angle=view_angle, head_angle=head_angle
)

visible_state_flat.shape

(41,)

### Step 

`step(dt)` is a method call on an instance of the Simulation class, where `dt` is a scalar that represents the length of each simulation timestep in seconds. It advances the simulation by one timestep, which can result in changes to the state of the simulation (for example, new positions of objects, updated velocities, etc.) based on the physical laws and rules defined in the simulation.

In the Waymo dataset, the length of the expert data is 9 seconds, and we use a step size of 0.1. The first second is used as a warm-start, leaving the remaining 8 seconds (80 steps) for the simulation (details in Section 3.3).

In [23]:
dt = 0.1

# Step the simulation
sim.step(dt)

### Action space

The action set for a vehicle consists of three components: acceleration, steering and the head angle. Actions are discretized based on a provided upper and lower bound.

In the experiments in the paper we use:
- 6 discrete actions for **acceleration** uniformly split between $[-3, 2] \, \frac{m}{s^2}$
- 21 discrete actions for **steering** between $[-0.7, 0.7]$ radians 
- 5 discrete actions for **head tilt** between $[-1.6, 1.6]$ radians

Here is you can access the action for a vehicle Nocturne:

In [33]:
# Choose an arbitrary timepoint
time = 5

# Show expert action at timepoint
scenario.expert_action(ego_vehicle, time)

{acceleration: -0.224648, steering: -0.360994, head_angle: 0.000000}

In [34]:
type(scenario.expert_action(ego_vehicle, time))

nocturne_cpp.Action

In [35]:
# How did the vehicle's position change after taking this action?
scenario.expert_pos_shift(ego_vehicle, time)

(-0.005859, 0.004639)

In [36]:
# How did the head angle change?
scenario.expert_heading_shift(ego_vehicle, time)

-0.0007097125053405762